# Modelo ADA con hiperparametros
 

### Libraires ###

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV




c:\Users\Victor\Anaconda3\envs\ST\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Funciones

In [2]:
def evaluate_model(y_test, pred, pred_proba = None):
    if pred_proba is not None:
        print('ROC-AUC score of the model: {}'.format(roc_auc_score(y_test, pred_proba[:, 1])))
    print('Accuracy of the model: {}\n'.format(accuracy_score(y_test, pred)))
    print('Classification report: \n{}\n'.format(classification_report(y_test, pred)))

### Division de los datos en train, test y validacion

In [3]:
new_df_base = pd.read_csv("../data/new_df_base.csv")
new_df_base.head()


,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,...,housing_status_BE,housing_status_BF,housing_status_BG,source_INTERNET,source_TELEAPP,device_os_linux,device_os_macintosh,device_os_other,device_os_windows,device_os_x11
0,1,0.9,0.166828,NaN,88.0,50,0.020925,NaN,769,10650.765523,...,0,0,0,1,0,0,0,0,1,0
1,1,0.9,0.296286,NaN,144.0,50,0.005418,NaN,366,534.047319,...,0,0,0,1,0,0,0,0,1,0
2,1,0.9,0.044985,NaN,132.0,40,3.108549,NaN,870,4048.534263,...,0,0,0,1,0,0,0,1,0,0
3,1,0.9,0.159511,NaN,22.0,50,0.019079,NaN,810,3457.064063,...,0,0,0,1,0,1,0,0,0,0
4,1,0.9,0.596414,NaN,218.0,50,0.004441,NaN,890,5020.341679,...,0,0,0,1,0,0,1,0,0,0


In [4]:
# Separate the feature matrix and target variable
X = new_df_base.drop(['fraud_bool'], axis=1)
y = new_df_base['fraud_bool']

# Impute missing values with mean
X = X.apply(lambda x: x.fillna(x.mean()))

In [5]:
# Dropping constant features from Variance Threshold test
X.drop(['device_fraud_count'], axis=1, inplace=True)

# # Selecting features from ExtraTreesClassifier test
# selected_features = [feature.replace('scaled__', '') for feature in best_extra_features]
# X = X[selected_features]

In [6]:
X_train_0, X_test, y_train_0, y_test = train_test_split(X, y, test_size=0.2, random_state=12)


In [7]:
# Crear el objeto RandomUnderSampler con la proporción deseada
under_sampler = RandomUnderSampler(sampling_strategy=0.1, random_state=12)

# Aplicar undersampling a tus datos
X_train_2, y_train_2 = under_sampler.fit_resample(X_train_0, y_train_0)

# Verificar la proporción después del undersampling
print('Undersampled dataset shape %s' % Counter(y_train_2))


Undersampled dataset shape Counter({0: 88940, 1: 8894})


### Entrenamiento del modelo con hiperparametros

In [8]:
ada_model = Pipeline([
    ('classifier', AdaBoostClassifier(random_state=12))
])

In [9]:
# Define a dictionary of hyperparameters for ADA Classifier
ada_params = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__base_estimator': [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2)],
    'classifier__algorithm': ['SAMME', 'SAMME.R']
}

CV= RandomizedSearchCV(estimator=ada_model, param_distributions=ada_params, cv=3, n_jobs=-1, verbose=2)

                                      

In [10]:
ADA_models=CV.fit(X_train_2, y_train_2)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [11]:
# Lo guardamos en un archivo pickle

#with open('../models/ADA_model_01.pkl', 'wb') as file:
#    pickle.dump(ADA_models, file)

In [12]:
# Cargamos el modelo ADA
with open('../models/ADA_model_01.pkl', 'rb') as file:
    ADA_model = pickle.load(file)

# Prediccion con el modelo
pred_ada = ADA_model.best_estimator_.predict(X_test)

# Prediccion de probabilidades con el modelo
pred_proba_ada = ADA_model.best_estimator_.predict_proba(X_test)

In [13]:
# Evaluación del modelo
evaluate_model(y_test, pred_ada)

Accuracy of the model: 0.98037

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    197865
           1       0.20      0.27      0.23      2135

    accuracy                           0.98    200000
   macro avg       0.59      0.63      0.61    200000
weighted avg       0.98      0.98      0.98    200000


